## 嵌入

在我们之前的示例中，我们对长度为“vocab_size”的高维词袋向量进行了操作，并且我们明确地将低维位置表示向量转换为稀疏的 one-hot 表示。 这种 one-hot 表示不是内存有效的，此外，每个单词都是相互独立处理的，即 one-hot 编码的向量不表示单词之间的任何语义相似性。

在本单元中，我们将继续探索 **News AG** 数据集。 首先，让我们加载数据并从上一个单元中获取一些定义。


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)
print("Vocab size = ",vocab_size)

Loading dataset...
Building vocab...
Vocab size =  95812


### 什么是嵌入？

**embedding** 的思想是通过低维密集向量来表示单词，以某种方式反映单词的语义。我们稍后将讨论如何构建有意义的词嵌入，但现在让我们将嵌入视为一种降低词向量维数的方法。

因此，嵌入层会将一个词作为输入，并产生一个指定“embedding_size”的输出向量。从某种意义上说，它与“线性”层非常相似，但它不是采用单热编码向量，而是能够将单词编号作为输入。

通过使用嵌入层作为我们网络的第一层，我们可以从 bag-or-words 切换到 **embedding bag** 模型，我们首先将文本中的每个单词转换为相应的嵌入，然后计算一些聚合函数所有这些嵌入，例如“sum”、“average”或“max”。

![显示五个序列词的嵌入分类器的图像。](./images/embedding-classifier-example.png)

我们的分类器神经网络将从嵌入层开始，然后是聚合层，最后是线性分类器

In [2]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x,dim=1)
        return self.fc(x)

### 处理可变序列大小

由于这种架构，我们网络的小批量需要以某种方式创建。 在前面的单元中，当使用词袋时，无论文本序列的实际长度如何，小批量中的所有 BoW 张量都具有相同的大小“vocab_size”。 一旦我们转向词嵌入，我们最终会在每个文本样本中得到可变数量的词，当将这些样本组合成小批量时，我们将不得不应用一些填充。

这可以使用向数据源提供 `collate_fn` 函数的相同技术来完成：


In [3]:
def padify(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # first, compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)

### 训练嵌入分类器

现在我们已经定义了合适的数据加载器，我们可以使用我们在上一单元中定义的训练函数来训练模型：

In [4]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=1, epoch_size=25000)

3200: acc=0.64125
6400: acc=0.68671875
9600: acc=0.7088541666666667
12800: acc=0.725859375
16000: acc=0.7340625
19200: acc=0.74328125
22400: acc=0.7511160714285714


(0.9622340382327655, 0.7568777991042867)

> **注意**：为了节省时间，我们这里只训练了 25k 条记录（少于一个完整的 epoch），但是你可以继续训练，写一个函数来训练几个 epoch，并用学习率参数来实验 达到更高的精度。 你应该能够达到大约 90% 的准确率。

### EmbeddingBag 层和可变长度序列表示

在之前的架构中，我们需要将所有序列填充到相同的长度，以便将它们放入一个小批量中。这不是表示可变长度序列的最有效方法 - 另一种方法是使用 **offset** 向量，它将保存存储在一个大向量中的所有序列的偏移量。

![显示偏移序列表示的图像](./images/offset-sequence-representation.png)

> **注意**：在上图中，我们显示了一个字符序列，但在我们的示例中，我们使用的是单词序列。然而，用偏移向量表示序列的一般原理保持不变。

为了使用偏移表示，我们使用 [`EmbeddingBag`](https://pytorch.org/docs/stable/generated/torch.nn.EmbeddingBag.html) 层。它类似于“Embedding”，但它以内容向量和偏移向量作为输入，还包括平均层，可以是“mean”、“sum”或“max”。

这是使用“EmbeddingBag”的修改后的网络：


In [5]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.EmbeddingBag(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, text, off):
        x = self.embedding(text, off)
        return self.fc(x)

为了准备训练数据集，我们需要提供一个转换函数来准备偏移向量：

In [6]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1])) for t in b]
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)

请注意，与之前所有示例不同的是，我们的网络现在接受两个参数：数据向量和偏移向量，它们的大小不同。 有趣的是，我们的数据加载器还为我们提供了 3 个值而不是 2 个：文本和偏移向量都作为特征提供。 因此，我们需要稍微调整我们的训练函数来解决这个问题：

In [7]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)

def train_epoch_emb(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.CrossEntropyLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    loss_fn = loss_fn.to(device)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,text,off in dataloader:
        optimizer.zero_grad()
        labels,text,off = labels.to(device), text.to(device), off.to(device)
        out = net(text, off)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count


train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6284375
6400: acc=0.68171875
9600: acc=0.7063541666666666
12800: acc=0.7259375
16000: acc=0.740125
19200: acc=0.74875
22400: acc=0.7557142857142857


(22.57764265435061, 0.7588371721049264)

## 语义嵌入：Word2Vec

在我们前面的例子中，模型嵌入层学会了将单词映射到向量表示，然而，这种表示没有太多的语义意义。学习这样的向量表示会很好，相似的词或符号将对应于在某些向量距离（例如欧几里得距离）方面彼此接近的向量。

为此，我们需要以特定方式在大量文本上预训练我们的嵌入模型。训练语义嵌入的第一种方法称为 [Word2Vec](https://en.wikipedia.org/wiki/Word2vec)。它基于两个主要架构，用于生成单词的分布式表示：

 - **连续词袋**（CBoW）——在这个架构中，我们训练模型从周围的上下文中预测一个词。给定 ngram $(W_{-2},W_{-1},W_0,W_1,W_2)$，模型的目标是从 $(W_{-2},W_{-1} ,W_1,W_2)$。
 - **Continuous skip-gram** 与 CBoW 相反。该模型使用上下文词的周围窗口来预测当前词。

CBoW 更快，而 skip-gram 更慢，但在表示不常用的词方面做得更好。

![显示 CBoW 和 Skip-Gram 算法将单词转换为向量的图像。](./images/example-algorithms-for-converting-words-to-vectors.png)

为了试验在 Google 新闻数据集上预训练的 word2vec 嵌入，我们可以使用 **gensim** 库。下面我们找到与“神经”最相似的词

> **注意：** 当您第一次创建词向量时，下载它们可能需要一些时间！

In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [9]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


我们还可以从单词中提取向量嵌入，用于训练分类模型（为了清晰起见，我们只显示了向量的前 20 个分量）：

In [10]:
w2v.word_vec('play')[:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

语义嵌入的好处在于您可以操纵向量编码来更改语义。 例如，我们可以要求找到一个单词，它的向量表示尽可能接近单词 *king* 和 *woman*，并且远离单词 *man*：

In [11]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

CBOW 和 Skip-Grams 都是“预测性”嵌入，因为它们只考虑本地上下文。 Word2Vec 不利用全局上下文。

**FastText**，通过学习每个单词的向量表示和在每个单词中找到的字符 n-gram 来构建在 Word2Vec 上。 然后在每个训练步骤中将表示的值平均为一个向量。 虽然这为预训练增加了许多额外的计算，但它使词嵌入能够对子词信息进行编码。

另一种方法**GloVe**利用共现矩阵的思想，使用神经方法将共现矩阵分解为更具表现力的非线性词向量。

您可以通过将嵌入更改为 FastText 和 GloVe 来使用示例，因为 gensim 支持

## 在 PyTorch 中使用预训练嵌入

我们可以修改上面的示例，以使用语义嵌入（例如 Word2Vec）预先填充嵌入层中的矩阵。 我们需要考虑到预训练嵌入的词汇表和我们的文本语料库可能不匹配，因此我们将使用随机值初始化缺失词的权重：

In [12]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

net = EmbedClassifier(vocab_size,embed_size,len(classes))

print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab.itos):
    try:
        net.embedding.weight[i].data = torch.tensor(w2v.get_vector(w))
        found+=1
    except:
        net.embedding.weight[i].data = torch.normal(0.0,1.0,(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")
net = net.to(device)

Embedding size: 300
Populating matrix, this will take some time...Done, found 41080 words, 54732 words missing


现在让我们训练我们的模型。 请注意，由于更大的嵌入层尺寸以及更多的参数，训练模型所需的时间明显大于前一个示例。 此外，因此，如果我们想避免过度拟合，我们可能需要在更多示例上训练我们的模型。

In [13]:
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6471875
6400: acc=0.68453125
9600: acc=0.7113541666666666
12800: acc=0.729375
16000: acc=0.7450625
19200: acc=0.7559895833333333
22400: acc=0.7624553571428572


(224.23208573256557, 0.7666346769033909)

在我们的例子中，我们没有看到准确度的巨大提高，这可能是完全不同的声音。
为了克服不同词汇的问题，我们可以使用以下解决方案之一：
* 根据我们的词汇重新训练 word2vec 模型
* 使用预训练的 word2vec 模型中的词汇加载我们的数据集。 可以在加载期间指定用于加载数据集的词汇。

后一种方法似乎更容易，特别是因为 PyTorch `torchtext` 框架包含对嵌入的内置支持。 例如，我们可以通过以下方式实例化基于 GloVe 的词汇表：

In [14]:
vocab = torchtext.vocab.GloVe(name='6B', dim=50)

.vector_cache/glove.6B.zip: 862MB [06:08, 2.34MB/s]                               
100%|█████████▉| 399999/400000 [00:06<00:00, 59733.65it/s]


加载的词汇有以下基本操作：
* `vocab.stoi` 字典允许我们将单词转换为其字典索引
* `vocab.itos` 做相反的事情 - 将数字转换为单词
* `vocab.vectors` 是嵌入向量的数组，因此要获得单词 `s` 的嵌入，我们需要使用 `vocab.vectors[vocab.stoi[s]]`

这是操纵嵌入的示例以演示等式 **kind-man+woman = Queen**（我不得不稍微调整系数以使其起作用）：

In [15]:
# get the vector corresponding to kind-man+woman
qvec = vocab.vectors[vocab.stoi['king']]-vocab.vectors[vocab.stoi['man']]+1.3*vocab.vectors[vocab.stoi['woman']]
# find the index of the closest embedding vector 
d = torch.sum((vocab.vectors-qvec)**2,dim=1)
min_idx = torch.argmin(d)
# find the corresponding word
vocab.itos[min_idx]

'queen'

要使用这些嵌入训练分类器，我们首先需要使用 GloVe 词汇对我们的数据集进行编码：

In [16]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1],voc=vocab)) for t in b] # pass the instance of vocab to encode function!
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

正如我们在上面看到的，所有向量嵌入都存储在 `vocab.vectors` 矩阵中。 使用简单的复制将这些权重加载到嵌入层的权重中变得非常容易：

In [17]:
net = EmbedClassifier(len(vocab),len(vocab.vectors[0]),len(classes))
net.embedding.weight.data = vocab.vectors
net = net.to(device)

现在让我们训练我们的模型，看看是否能得到更好的结果：

In [18]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6396875
6400: acc=0.68375
9600: acc=0.7080208333333333
12800: acc=0.721796875
16000: acc=0.73375
19200: acc=0.7439583333333334
22400: acc=0.7525892857142857


(35.61183671225208, 0.7571976967370442)

我们没有看到准确率显着提高的原因之一是由于我们数据集中的某些单词在预训练的 GloVe 词汇表中缺失，因此它们基本上被忽略了。 为了克服这个事实，我们可以在我们的数据集上训练我们自己的嵌入。


## 训练你自己的嵌入

在我们的示例中，我们一直在使用预训练的语义嵌入，但看看如何使用 CBoW 或 Skip-gram 架构训练这些嵌入很有趣。 这个练习超出了这个模块，但那些有兴趣的人可能想看看这个[关于语言建模的官方 PyTorch 教程](https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html)。 此外，**gensim** 框架可用于在几行代码中训练最常用的嵌入，如本文档中所述（https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html） .

## 上下文嵌入

Word2Vec 等传统预训练嵌入表示的一个关键限制是词义消歧问题。虽然预训练嵌入可以捕获上下文中单词的某些含义，但单词的每个可能含义都被编码到相同的嵌入中。这可能会导致下游模型出现问题，因为许多单词（例如“play”）根据使用的上下文具有不同的含义。

例如，这两个不同句子中的“play”一词具有完全不同的含义：
- 我去剧院看了一场**剧**。
- 约翰想和他的朋友们**玩**。

上面的预训练嵌入在同一个嵌入中代表了单词“play”的这两种含义。为了克服这个限制，我们需要基于 **语言模型** 构建嵌入，该模型在大型文本语料库上训练，并且*知道*如何将单词放在不同的上下文中。讨论上下文嵌入超出了本教程的范围，但我们将在下一个单元中讨论语言模型时回到它们。
